In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline

sns.set_style("whitegrid")

In [2]:
fashion_mnist = input_data.read_data_sets("data/fashion", source_url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/fashion\t10k-images-idx3-ubyte.gz
Extracting data/fashion\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def crear_placeholders(numero_x, numero_y):
    x = tf.placeholder(tf.float32, shape=[None, numero_x])
    y = tf.placeholder(tf.float32, shape=[None, numero_y])
    
    return x, y

In [4]:
def inicializar_parametros():
    
    # Primera capa
    w1 = tf.get_variable("w1", [784, 128], initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.get_variable("b1", [1, 128], initializer=tf.zeros_initializer())
    
    # Segunda capa
    w2 = tf.get_variable("w2", [128, 128], initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.get_variable("b2", [1, 128], initializer=tf.zeros_initializer())
    
    # Capa de salida
    w3 = tf.get_variable("w3", [128, 10], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.get_variable("b3", [1, 10], initializer=tf.zeros_initializer())
    
    parametros = {
        "w1": w1,
        "b1": b1,
        "w2": w2,
        "b2": b2,
        "w3": w3,
        "b3": b3
    }
    
    return parametros  

In [5]:
def forward_propagation(x, parametros):
    
    w1 = parametros['w1']
    b1 = parametros['b1']
    w2 = parametros['w2']
    b2 = parametros['b2']
    w3 = parametros['w3']
    b3 = parametros['b3']
    
    z1 = tf.add(tf.matmul(x, w1), b1)
    a1 = tf.nn.relu(z1)                  
    z2 = tf.add(tf.matmul(a1, w2), b2)    
    a2 = tf.nn.relu(z2)                  
    z3 = tf.add(tf.matmul(a2, w3), b3)
    
    return z3

In [6]:
def compute_loss(z, y_true):
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=z, labels=y_true))
    
    return loss

In [7]:
def optimize(loss, learning_rate=0.001):
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    return optimizer

# Inicializamos todo

In [8]:
tf.reset_default_graph()

(cantidad_de_x, nuestra_x) = fashion_mnist.train.images.shape
nuestra_y = fashion_mnist.train.labels.shape[1]    
    
x, y = crear_placeholders(nuestra_x, nuestra_y)
    
parametros = inicializar_parametros()
    
z3 = forward_propagation(x, parametros)
    
loss = compute_loss(z3, y)
    
train = optimize(loss)

In [9]:
init = tf.global_variables_initializer()

## Creamos nuestra session

In [10]:
with tf.Session() as sess:
    
    sess.run(init)
    
    batchsize = 32
    
    cantidad_loops = int(cantidad_de_x / batchsize)
    
    # Entrenamos el modelo
    for step in range(cantidad_loops):
        
        batch_x, batch_y = fashion_mnist.train.next_batch(batchsize)
        
        sess.run(train, feed_dict={x: batch_x, y: batch_y})
        
    correct_prediction = tf.equal(tf.argmax(z3, 1), tf.argmax(y, 1))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # Testamos nuestro model con datos que no ha visto
    print(sess.run(accuracy, feed_dict={x: fashion_mnist.test.images, y: fashion_mnist.test.labels}))

0.8498
